# Objective:
    To read QR code/barcode and use it for logging in the system.

# Key Features:
    Can detect multiple Barcode/ QR code in the same frame
    Capability of Working with webcams
    Database interacation for logging IN

In [1]:
from  pyzbar import pyzbar
import cv2 as cv
import mysql.connector

In [2]:
class Logging():
    '''
    To get data from the database
    Build to Encapsulate the data
    Also compare the decoded text with the database data
    '''
    def __init__(self, filePath):
        self.data = None
        
        dit = {}
        file = open(filePath)
        for data in file:
            data = data[:-1]
            key, value = data.split(':')
            dit[key] = value

        #Connecting to the database
        self.mydb = mysql.connector.connect(host=dit['host'], user=dit['user'], password=dit['password'], database=dit['database'])
        
        #Deleting the database access details
        del dit
        
    def get_password(self):
        mycursor = mydb.cursor()
        mycursor.execute('select * from logging;')
        self.data = mycursor.fetchall()
        self.mydb.close()
    
    def check_password(self, password):
        if self.data == None:
            self.get_password()
        
        if password in self.data:
            return True
        return False       

In [3]:
def decode_image(image):
    '''
    Finds Barcode/QR code in the image and draws binding boxes around them
    And also writes the decoded data in the image
    input image should be an array form
    '''
    #Finding the Barcode/QR code
    barcodes = pyzbar.decode(image)
    for barcode in barcodes:
        #Creating a bounding box around the barcode
        x, y, w, h = barcode.rect
        cv.rectangle(image, (x, y), (x + w, y + h), (0, 0, 255), 2)

        #Adding decoded text to the bounding box
        text = barcode.data.decode("utf-8") + ' (' + barcode.type+ ')'
        if y==0:
            y = y+20
        cv.putText(image, text, (x, y-10), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)
        #print("[INFO]\n{} ".format(text))

In [4]:
#File contains the database access details
filePath = r'C:\Users\rspai\Downloads\database.txt'
log = Logging(filePath)

In [ ]:
#Intinializing the webcam to use
cap = cv.VideoCapture(0)

#Reading video as a series of images
while True:
    #Reading images from webcam
    _,frame = cap.read()
    cv.imshow('webcam', frame)
    
    #Finding the Barcode/QR code
    barcodes = pyzbar.decode(frame)
    
    #Checking if there is no barcode in the image
    if len(barcodes)==0:
        continue
        
    for barcode in barcodes:
        #Adding decoded text to the bounding box
        decoded = barcode.data.decode("utf-8")
        if log.check_password(decoded):
            welcome = cv.imread(r'C:\project\ng-fundamentals\src\assets\images\welcome.png')
            cv.imshow('welcome', welcome)
            cv.waitKey(0)
            break